In [12]:
import pandas as pd
import numpy as np
import requests
import urllib.parse
from urllib.parse import urlencode
import base64
import webbrowser
import time
import re
import json
from datasketch import MinHash
import matplotlib.pyplot as plt
from dotenv import load_dotenv

In [13]:
load_dotenv()
CLIENT_ID = os.getenv('ENV_CLIENT_ID')
CLIENT_SECRET = os.getenv('ENV_CLIENT_SECRET')

## Get spotify token 

In [14]:
AUTH_URL = 'https://accounts.spotify.com/authorize'
TOKEN_URL = 'https://accounts.spotify.com/api/token'
BASE_URL = 'https://api.spotify.com/v1/'
# SPOTIFY API CLIENT ID AND SECRET

In [31]:
auth_headers = {
    "client_id": CLIENT_ID,
    "response_type": "code",
    "redirect_uri": "http://localhost:7777/callback",
    "scope": "user-library-read"
}

webbrowser.open("https://accounts.spotify.com/authorize?" + urlencode(auth_headers))

True

---
### From the URL, copy everything after code= and put that into code in the cell below

---

In [32]:
code = 'AQD_noEN-jYd7N3_HmPsXfhnKKzK9OrByB6DOpGdfTtsz048oD00JGZhAzuozGVmrjRSB1thccJhFLIB2tN4jv3NHxzAP9JoCTzuJFTOHm4dZgQlAHUloe-4CvVxloykkm-Eovce0u3Vb0Vy2rsyRErCQpJRRMAnQZMHoEI52N_jFqnoxzPxDeTtypttISKs_HRriWM'
encoded_credentials = base64.b64encode(CLIENT_ID.encode() + b':' + CLIENT_SECRET.encode()).decode("utf-8")

token_headers = {
    "Authorization": "Basic " + encoded_credentials,
    "Content-Type": "application/x-www-form-urlencoded"
}

token_data = {
    "grant_type": "authorization_code",
    "code": code,
    "redirect_uri": "http://localhost:7777/callback"
}

r = requests.post("https://accounts.spotify.com/api/token", data=token_data, headers=token_headers)
ACCESS_TOKEN = r.json()['access_token']

In [33]:
user_headers = {
    "Authorization": "Bearer " + ACCESS_TOKEN,
    "Content-Type": "application/json"
}

ACCESS_TOKEN

'BQAfTbE3bTgj2xQ5zS9WxyzkvxN3M7jTpWO_4_gODpK6WeWKtxk5u23lDdoSvEpzL6TRMvWZqC9q4EuxKxLdOWXf1liLMQqd2leQdZAJ7FkNNbLFuL2xZxZNM5EA1tGMqyMCDd-4y0eGiTduiy7-h9-xNIGPBLE15olbFlc5QghzaTOSMdd7OE5y8C32gvU'

## Load spotify ids for kaggle data

In [19]:
spotify_data = pd.read_csv('../data/etl_integration/kaggle-spotify-2023.csv', encoding='latin-1')

In [20]:
output = []
failed = []
count_failed = 0
count = 0
for _, row in spotify_data.iterrows():
    print(count, end='\r')
    track_name = re.sub(r'[^a-zA-Z0-9_\-+]+', '', row['track_name'].replace(' ', '+'))
    artist_name = re.sub(r'[^a-zA-Z0-9_\-+]+', '', row['artist(s)_name'].replace(' ', '+'))
    #query = f'track%3A{track_name}%2520artist%3A{artist_name}'
    query = f'track%253A{track_name}%20artist3A{artist_name}'
    resp = requests.get(f'https://api.spotify.com/v1/search?q={query}&type=track&limit=5', headers=user_headers)
    track_id = -1
    if resp.status_code == 200:
        for i, track in enumerate(resp.json()['tracks']['items']):
            if track['name'] == re.sub(r'[^a-zA-Z0-9_\-+]+', '', track_name.replace(' ', '+')):
                track_id = i
        if track_id == -1:
            namelist = []
            for track in resp.json()['tracks']['items']:
                namelist.append(re.sub(r'[^a-zA-Z0-9_\-+]+', '', track['name'].replace(' ', '+')))
            sim_ratings = []
            track_split = set(track_name.split('+'))
            for i, trackname in enumerate(namelist):
                sim_ratings.append(len(set(trackname.split('+')).symmetric_difference(track_split)))
            #popularities = [track['popularity'] for track in resp.json()['tracks']['items']]
            #track_id = popularities.index(max(popularities))
            track_id = sim_ratings.index(min(sim_ratings))
        output.append((resp, resp.json()['tracks']['items'][track_id], resp.json()))
        count_failed = 0
    else:
        failed.append(resp)
        count_failed += 1
    if count_failed == 5:
        break
    count += 1

In [21]:
trackids = []
for track in output:
    trackids.append(track[1]['id'])

## Check for correctness

In [22]:
track_jaccards = []
artist_jaccards = []
api_tracks = []
api_artists = []
track_ids = []
def get_string_jaccard(s1, s2):
    m1, m2 = MinHash(), MinHash()
    for d in s1.split():
        m1.update(d.encode('utf8'))
    for d in s2.split():
        m2.update(d.encode('utf8'))
    return m1.jaccard(m2)
for i, row in spotify_data.iterrows():
    data_tname = re.sub('[^0-9a-zA-Z ]+', '', row['track_name'])
    data_artists = re.sub('[^0-9a-zA-Z ]+', '', row['artist(s)_name'])
    track_items = output[i][2]['tracks']['items']
    scores = []
    for api_track in track_items:
        api_artists_joined = ', '.join([artist['name'] for artist in api_track['artists']])
        api_tname = re.sub('[^0-9a-zA-Z ]+', '', api_track['name'])
        api_tartists = re.sub('[^0-9a-zA-Z ]+', '', api_artists_joined)
        track_jaccard = get_string_jaccard(data_tname.lower().strip(), api_tname.lower().strip())
        artist_jaccard = get_string_jaccard(data_artists.lower().strip(), api_tartists.lower().strip())
        scores.append((track_jaccard, artist_jaccard))
    multiplied_scores = list(map(lambda x: x[0] * x[1], scores))
    best_match_idx = multiplied_scores.index(max(multiplied_scores))
    track_jaccards.append(scores[best_match_idx][0])
    artist_jaccards.append(scores[best_match_idx][1])
    api_tracks.append(track_items[best_match_idx]['name'])
    api_artists.append(', '.join([artist['name'] for artist in track_items[best_match_idx]['artists']]))
    track_ids.append(track_items[best_match_idx]['id'])
spotify_data.insert(0, 'track_id', track_ids)
spotify_data['api_track_name'] = api_tracks
spotify_data['api_artist(s)'] = api_artists
spotify_data['track_jaccards'] = track_jaccards
spotify_data['artist_jaccards'] = artist_jaccards

In [23]:
spotify_data['api_confidence'] = spotify_data['track_jaccards'].multiply(spotify_data['artist_jaccards'])

In [24]:
filtered_spotify_data = spotify_data[spotify_data['api_confidence'] >= 0.25]

In [29]:
filtered_spotify_data.to_csv('../data/etl_integration/kaggle_spotify_data_with_id.csv', index=False)

In [27]:
spotify_data[spotify_data['api_confidence'] == 1][['track_name', 'artist(s)_name', 'api_track_name', 'api_artist(s)']]

,track_name,artist(s)_name,api_track_name,api_artist(s)
0,Seven (feat. Latto) (Explicit Ver.),"Latto, Jung Kook",Seven (feat. Latto) (Explicit Ver.),"Jung Kook, Latto"
1,LALA,Myke Towers,LALA,Myke Towers
2,vampire,Olivia Rodrigo,vampire,Olivia Rodrigo
3,Cruel Summer,Taylor Swift,Cruel Summer,Taylor Swift
5,Sprinter,"Dave, Central Cee",Sprinter,"Dave, Central Cee"
...,...,...,...,...
945,BackOutsideBoyz,Drake,BackOutsideBoyz,Drake
946,Broke Boys,"Drake, 21 Savage",Broke Boys,"Drake, 21 Savage"
948,My Mind & Me,Selena Gomez,My Mind & Me,Selena Gomez
950,A Veces (feat. Feid),"Feid, Paulo Londra",A Veces (feat. Feid),"Paulo Londra, Feid"


In [28]:
spotify_data[spotify_data['api_confidence'] == 0][['track_name', 'artist(s)_name', 'api_track_name', 'api_artist(s)']]

,track_name,artist(s)_name,api_track_name,api_artist(s)
4,WHERE SHE GOES,Bad Bunny,Where She Goes (Remix),Motoroladovi
11,Super Shy,NewJeans,Super Shy - NewJeans (Marimba Ringtone),"Anime Ringtones, Anytunz"
14,As It Was,Harry Styles,As It Was (Originally Performed by Harry Style...,Vox Freaks
17,What Was I Made For? [From The Motion Picture ...,Billie Eilish,What Was I Made For? (Epilogue) - Instrumental...,"Mark Ronson, Andrew Wyatt"
19,Like Crazy,Jimin,Arrivals n2,Dustin O'Halloran
...,...,...,...,...
941,"Would've, Could've, Should've",Taylor Swift,"Would've, Could've, Should've - Piano Version",Marlene T
943,Privileged Rappers,"Drake, 21 Savage",Privileged Rappers (Originally Performed by Dr...,3 Dope Brothas
947,The Great War,Taylor Swift,The Great War (Originally Performed by Taylor ...,Sing2Piano
949,Bigger Than The Whole Sky,Taylor Swift,Bigger Than The Whole Sky - Piano Version,Marlene T


---

As we can see above, entries where the confidence is 1 have the correct track name and artist(s), and tracks with confidence 0 are all incorrect

---

## Get track info and audio features

In [34]:
audio_features = ['acousticness', 'danceability', 'duration_ms', 'energy', 'instrumentalness', 'key', 'liveness',
                  'loudness', 'mode', 'speechiness', 'tempo', 'time_signature', 'valence']
track_data = {col: [] for col in ['track_id', 'track_name', 'artist(s)', 'related_artists'] + audio_features}
failed = []
failed_artists = []
track_audio_features = []
track_audio_analysis = []
for i, track_id in enumerate(filtered_spotify_data['track_id'].tolist()):
    print(round(i / len(spotify_data), 4), end='\r')
    track_resp = requests.get(f'https://api.spotify.com/v1/tracks/{track_id}', headers=user_headers)
    track_json = None
    if track_resp.status_code == 200:
        track_json = track_resp.json()
        artists = [(artist['name'], artist['id']) for artist in track_json['artists']]
        related_artists = []
        for artist in artists:
            artist_id = artist[1]
            related_artists_resp = requests.get(f'https://api.spotify.com/v1/artists/{artist_id}/related-artists',
                                                headers=user_headers)
            related_artists_json = None
            if related_artists_resp.status_code == 200:
                related_artists_json = related_artists_resp.json()
                related_artists.append([(related_artist['name'], related_artist['id']) for related_artist in related_artists_json['artists']])
            else:
                failed_artists.append((artist, related_artists_resp))
                related_artists.append(None)
        track_data['track_id'].append(track_id)
        track_data['track_name'].append(track_json['name'])
        track_data['artist(s)'].append(artists)
        track_data['related_artists'].append(related_artists)
    else:
        failed.append(('track', track_id, track_resp))
    # TRACK AUDIO FEATURES
    track_features_resp = requests.get(f'https://api.spotify.com/v1/audio-features/{track_id}', headers=user_headers)
    track_features_json = None
    if track_features_resp.status_code == 200:
        track_features_json = track_features_resp.json()
        for feature in audio_features:
            if feature in track_features_json:
                track_data[feature].append(track_features_json[feature])
            else:
                track_data[feature].append(np.nan)
        track_audio_features.append(track_features_json)
    else:
        failed.append(('audio-features', track_id, track_features_resp))
    # TRACK AUDIO ANALYSIS
    track_analysis_resp = requests.get(f'https://api.spotify.com/v1/audio-analysis/{track_id}', headers=user_headers)
    track_analysis_json = None
    if track_analysis_resp.status_code == 200:
        track_analysis_json = track_analysis_resp.json()
        track_audio_analysis.append(track_analysis_json)
    else:
        failed.append(('audio-analysis', track, track_analysis_resp))
print()
track_data = pd.DataFrame(track_data)

0.8038


ValueError: All arrays must be of the same length

In [ ]:
with open('../data/etl_integration/kaggle_track_audio_features.json', 'w') as f:
    json.dump(track_audio_features, f)
with open('../data/etl_integration/kaggle_track_audio_analysis.json', 'w') as f:
    json.dump(track_audio_analysis, f)
track_data.to_csv('../data/etl_integration/kaggle_spotify_track_data.csv', index=False)